# IMPORTS

In [133]:
import pandas as pd
import numpy as np
import csv
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array
import itertools
from pickle import load

from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding

from pickle import load
from numpy import array
from numpy import argmax
import tensorflow as tf
from keras.models import load_model

from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split

from tf.keras.utils import register_keras_serializable
from tensorflow_addons import seq2seq

print("Done")

ImportError: cannot import name 'register_keras_serializable' from 'tensorflow.keras.utils' (unknown location)

# DATA CLEANING

In [135]:
df = pd.read_csv("sample.csv") # replace with "sample.csv" after training

del df['inbound']
del df['created_at']

df['response'] = ""


for line in df.index:
    
    g = df.loc[line, 'text'].lower()
    g = g.split()
    g = g[1:]
    g = " ".join(g)
    df.loc[line, 'text'] = g
    
    
    for line2 in df.index:
        
        if str(df.loc[line2, 'response_tweet_id']) == str(df.loc[line, 'tweet_id']):
            
            l = df.loc[line, 'text'].lower()
            l = l.split()
            #l = l[1:]
            l = " ".join(l)
            df.loc[line2, 'response'] = l
            
            break
            
for line in df.index:
    
    if len(str(df.loc[line, 'author_id'])) > 7 or df.loc[line, 'author_id'] == 'Tesco':
        df.drop(line, inplace = True)

        
df = df.dropna()
            
#df.head(20)   

# TOKENIZATION

In [136]:
text_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = "")
text_tokenizer.fit_on_texts(df['text'])

resp_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters = "")
resp_tokenizer.fit_on_texts(df['response'])

text_data = text_tokenizer.texts_to_sequences(df['text'])
text_data = tf.keras.preprocessing.sequence.pad_sequences(text_data, padding = 'post')

resp_data = resp_tokenizer.texts_to_sequences(df['response'])
resp_data = tf.keras.preprocessing.sequence.pad_sequences(resp_data, padding = 'post')

def max_len(tensor):
    return max( len(t) for t in tensor)

# PARAMETERS

In [137]:
X_train,  X_test, Y_train, Y_test = train_test_split(df['text'],df['response'],test_size=0.1)
BATCH_SIZE = 64
BUFFER_SIZE = len(X_train)
steps_per_epoch = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 256
rnn_units = 1024
dense_units = 1024
Dtype = tf.float32   #used to initialize DecoderCell Zero state

# DATA PREP

In [139]:
Tx = max_len(df['text'])
Ty = max_len(df['response'])

input_vocab_size = len(text_tokenizer.word_index)+1
output_vocab_size = len(resp_tokenizer.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder = True)

example_X, example_Y = next(iter(dataset))

print(example_X.shape)
print(example_Y.dtype)

(64,)
<dtype: 'string'>


# BUILDING MODEL

In [129]:
class EncoderNetwork(tf.keras.Model):
    def __init__(self,input_vocab_size,embedding_dims, rnn_units ):
        super().__init__()
        self.encoder_embedding = tf.keras.layers.Embedding(input_dim=input_vocab_size,
                                                           output_dim=embedding_dims)
        self.encoder_rnnlayer = tf.keras.layers.LSTM(rnn_units,return_sequences=True, 
                                                     return_state=True )
    
#DECODER
class DecoderNetwork(tf.keras.Model):
    def __init__(self,output_vocab_size, embedding_dims, rnn_units):
        super().__init__()
        self.decoder_embedding = tf.keras.layers.Embedding(input_dim=output_vocab_size,
                                                           output_dim=embedding_dims) 
        self.dense_layer = tf.keras.layers.Dense(output_vocab_size)
        self.decoder_rnncell = tf.keras.layers.LSTMCell(rnn_units)
        # Sampler
        self.sampler = tfa.seq2seq.sampler.TrainingSampler()
        # Create attention mechanism with memory = None
        self.attention_mechanism = self.build_attention_mechanism(dense_units,None,BATCH_SIZE*[Tx])
        self.rnn_cell =  self.build_rnn_cell(BATCH_SIZE)
        self.decoder = tfa.seq2seq.BasicDecoder(self.rnn_cell, sampler= self.sampler,
                                                output_layer=self.dense_layer)

    def build_attention_mechanism(self, units,memory, memory_sequence_length):
        return tfa.seq2seq.LuongAttention(units, memory = memory, 
                                          memory_sequence_length=memory_sequence_length)
        #return tfa.seq2seq.BahdanauAttention(units, memory = memory, memory_sequence_length=memory_sequence_length)

    # wrap decodernn cell  
    def build_rnn_cell(self, batch_size ):
        rnn_cell = tfa.seq2seq.AttentionWrapper(self.decoder_rnncell, self.attention_mechanism,
                                                attention_layer_size=dense_units)
        return rnn_cell
    
    def build_decoder_initial_state(self, batch_size, encoder_state,Dtype):
        decoder_initial_state = self.rnn_cell.get_initial_state(batch_size = batch_size, 
                                                                dtype = Dtype)
        decoder_initial_state = decoder_initial_state.clone(cell_state=encoder_state) 
        return decoder_initial_state

encoderNetwork = EncoderNetwork(input_vocab_size,embedding_dims, rnn_units)
decoderNetwork = DecoderNetwork(output_vocab_size,embedding_dims, rnn_units)
optimizer = tf.keras.optimizers.Adam()

NameError: name 'tfa' is not defined

# INITIALIZING TRAINING

In [ ]:
def loss_function(y_pred, y):
   
    #shape of y [batch_size, ty]
    #shape of y_pred [batch_size, Ty, output_vocab_size] 
    sparsecategoricalcrossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,
                                                                                  reduction='none')
    loss = sparsecategoricalcrossentropy(y_true=y, y_pred=y_pred)
    mask = tf.logical_not(tf.math.equal(y,0))   #output 0 for y=0 else output 1
    mask = tf.cast(mask, dtype=loss.dtype)
    loss = mask* loss
    loss = tf.reduce_mean(loss)
    return loss


def train_step(input_batch, output_batch,encoder_initial_cell_state):
    #initialize loss = 0
    loss = 0
    with tf.GradientTape() as tape:
        encoder_emb_inp = encoderNetwork.encoder_embedding(input_batch)
        a, a_tx, c_tx = encoderNetwork.encoder_rnnlayer(encoder_emb_inp, 
                                                        initial_state =encoder_initial_cell_state)

        #[last step activations,last memory_state] of encoder passed as input to decoder Network
        
         
        # Prepare correct Decoder input & output sequence data
        decoder_input = output_batch[:,:-1] # ignore <end>
        #compare logits with timestepped +1 version of decoder_input
        decoder_output = output_batch[:,1:] #ignore <start>


        # Decoder Embeddings
        decoder_emb_inp = decoderNetwork.decoder_embedding(decoder_input)

        #Setting up decoder memory from encoder output and Zero State for AttentionWrapperState
        decoderNetwork.attention_mechanism.setup_memory(a)
        decoder_initial_state = decoderNetwork.build_decoder_initial_state(BATCH_SIZE,
                                                                           encoder_state=[a_tx, c_tx],
                                                                           Dtype=tf.float32)
        
        #BasicDecoderOutput        
        outputs, _, _ = decoderNetwork.decoder(decoder_emb_inp,initial_state=decoder_initial_state,
                                               sequence_length=BATCH_SIZE*[Ty-1])

        logits = outputs.rnn_output
        #Calculate loss

        loss = loss_function(logits, decoder_output)

    #Returns the list of all layer variables / weights.
    variables = encoderNetwork.trainable_variables + decoderNetwork.trainable_variables  
    # differentiate loss wrt variables
    gradients = tape.gradient(loss, variables)

    #grads_and_vars – List of(gradient, variable) pairs.
    grads_and_vars = zip(gradients,variables)
    optimizer.apply_gradients(grads_and_vars)
    return loss

def initialize_initial_state():
        return [tf.zeros((BATCH_SIZE, rnn_units)), tf.zeros((BATCH_SIZE, rnn_units))]